In [74]:
!pip install -U datasets
!pip install dataclass

ERROR: Could not find a version that satisfies the requirement dataclass (from versions: none)
ERROR: No matching distribution found for dataclass


In [3]:
from google.colab import userdata
from datasets import load_dataset
import json

OPENROUTER_API_KEY = userdata.get('OPENROUTER_API_KEY')
model = "qwen/qwen3-8b:free"

In [3]:
num_cases = 50
reasoning_start = "<think>"
reasoning_end   = "</think>"

system_prompt = \
f"""You are given a competitive programming problem.

Your task is to write a Python script that generates test cases for this problem.

Requirements:
Read the problem description and extract the input format.

Identify the structure, types, and constraints of the inputs.

Understand any special cases or edge case patterns that should be tested.

Write a Python test case generator that:

Produces random, valid input data consistent with the problem’s input format and constraints.

Gives the final result in a string that follows the format of the problem input

Ensure the generated input is meaningful:

The input must be accepted by a correct solution to the problem and must not cause runtime errors.

Cover a variety of cases: small values, edge limits, and potentially tricky inputs that might expose bugs.

Do not write hardcoded input or expected output.

Your goal is to write a test case generator script, not fixed sample inputs or answers.

Output:
Your response should include a Python script that:

When run, generates and returns random valid inputs for the given problem.

Remember to store the outputs in a string or an array of strings

You shoud generate exactly {num_cases} test cases.

Output two new lines between each test case.


Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.

Provide your final generator program in this format:
```python
# YOUR VALIDATOR PROGRAM HERE
```
"""
print(system_prompt)

You are given a competitive programming problem.

Your task is to write a Python script that generates test cases for this problem.

Requirements:
Read the problem description and extract the input format.

Identify the structure, types, and constraints of the inputs.

Understand any special cases or edge case patterns that should be tested.

Write a Python test case generator that:

Produces random, valid input data consistent with the problem’s input format and constraints.

Gives the final result in a string that follows the format of the problem input

Ensure the generated input is meaningful:

The input must be accepted by a correct solution to the problem and must not cause runtime errors.

Cover a variety of cases: small values, edge limits, and potentially tricky inputs that might expose bugs.

Do not write hardcoded input or expected output.

Your goal is to write a test case generator script, not fixed sample inputs or answers.

Output:
Your response should include a Python s

In [4]:
ds = load_dataset("BAAI/TACO", split="train")
def filter_questions(ds):
  good_ds=[]
  bad_ds=[]
  keywords = [
    "interact with", "query the judge", "ask the judge", "exchange of data",
    "communicate back and forth", "judge responds", "your program will wait",
    "flush(", "adaptive input", "adversarial", "guess the number", "interactive"
  ]

  for i in range(len(ds)):
    question = ds[i]['question']
    if any(key in question for key in keywords):
      bad_ds.append(ds[i])
    else:
      good_ds.append(ds[i])
  return good_ds

ds=filter_questions(ds)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [1]:
dict2={}
for i in range(len(ds)):
  if ds[i]['source'] not in dict2:
    dict2[ds[i]['source']]=0
  dict2[ds[i]['source']]+=1
print(dict2)

NameError: name 'ds' is not defined

Interactive: {'codeforces': 113, 'kattis': 1, 'aizu': 3, 'atcoder': 1, 'codechef': 1}

Flush:{'codeforces': 133, 'codechef': 5, 'aizu': 8, 'kattis': 1, 'codewars': 1}

---



Interative and flush:{'codeforces': 138, 'codechef': 5, 'kattis': 2, 'aizu': 9, 'atcoder': 1, 'codewars': 1}

interatice, interaction, and flush: {'codeforces': 141, 'codechef': 9, 'hackerearth': 3, 'codewars': 3, 'kattis': 4, 'aizu': 9, 'atcoder': 1}

In [65]:
# Use OpenAI library as a base
from openai import OpenAI


client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY
)

def generate_completion(question):
    completion = client.chat.completions.create(
        model=model,
        temperature=1.0,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]
    )
    return completion.choices[0].message.content

message = generate_completion(question)
print(message)

```python
import random
import string

for _ in range(50):
    s_length = random.randint(1, 5000)
    t_length = random.randint(1, 5000)
    s = ''.join(random.choices(string.ascii_lowercase, k=s_length))
    t = ''.join(random.choices(string.ascii_lowercase, k=t_length))
    print(s)
    print(t)
    print()
```


In [66]:
import re

def extract_code(text: str):
    result = re.findall('```python(.*?)```', text, re.DOTALL)
    if len(result) > 0:
        return result[-1].rstrip()
    return None

code = extract_code(message)
print(code)


import random
import string

for _ in range(50):
    s_length = random.randint(1, 5000)
    t_length = random.randint(1, 5000)
    s = ''.join(random.choices(string.ascii_lowercase, k=s_length))
    t = ''.join(random.choices(string.ascii_lowercase, k=t_length))
    print(s)
    print(t)
    print()


In [67]:
from dataclasses import dataclass
from typing import Optional, List
import io
import time
import builtins
import sys
import multiprocessing
import resource

# Import libraries to make available for the test code that is generated
import math
import random
import numpy as np

default_globals = {
    "__builtins__": __builtins__,
    "math": math,
    "random": random,
    "np": np,
    "sys": sys
}

@dataclass
class CodeResult:
    """Result from executing a piece of code."""
    time: float  # seconds
    memory: float  # MB
    verdict: str  # "OK", "TLE", "MLE", "RTE", "WA"
    output: Optional[str] = None
    error: Optional[str] = None

def run_code(
    code: str,
    case_input: str,
    output_queue: multiprocessing.Queue,
    memory_limit: float = 256
) -> CodeResult:
    """Runs a piece of code with given inputs and captures the outputs"""

    #reliability_guard(memory_limit)

    def get_peak_memory_mb():
        """Get peak memory usage"""
        usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
        if sys.platform == 'darwin': # darwin is Mac OS X
            return usage / (1024 * 1024)  # Convert bytes -> MB
        else:
            return usage / 1024 # Convert KB -> MB

    start = time.time()

    # Mock input
    input_stream = io.StringIO(case_input)
    sys.stdin = input_stream
    builtins.input = lambda: input_stream.readline().rstrip('\n')  # Optional override

    # Capture output
    buf = io.StringIO()
    sys.stdout = buf
    try:
        exec(code, default_globals)
    except Exception as e:
        if (get_peak_memory_mb() >= memory_limit):
            output_queue.put(CodeResult(
                time=time.time() - start,
                memory=get_peak_memory_mb(),
                verdict="Memory Limit Error",
                error=str(e)
            ), 0)
            return

        output_queue.put(CodeResult(
            time=time.time() - start,
            memory=get_peak_memory_mb(),
            verdict="Runtime Error",
            error=str(e)
        ), 0)
        return
    finally:
        sys.stdout = sys.__stdout__

    result = CodeResult(
        output=buf.getvalue(),
        time=time.time() - start,
        memory=get_peak_memory_mb(),
        verdict="OK"
    )

    output_queue.put(result, 0)

def test_code(code: str, cases: List[str], time_limit: float = 256) -> List[CodeResult]:
    outs = []

    for i in range(len(cases)):
        output_queue = multiprocessing.Queue()
        p = multiprocessing.Process(target=run_code, args=(code, cases[i], output_queue))
        p.start()
        p.join(timeout=time_limit)
        if p.is_alive():
            p.terminate()
            out = CodeResult(
                time=time_limit,
                memory=0,
                verdict="Time Limit Error"
            )
        else:
            try:
                out = output_queue.get(timeout=1)
            except Exception as e:
                out = CodeResult(
                    time=0,
                    memory=0,
                    verdict="Runtime Error",
                    error=str(e)
                )

        outs.append(out)

    return outs

In [68]:
outs = test_code(code, ['andrew is smart'], 2)
if outs[0].verdict == 'OK':
  inputs=outs[0].output
  inputs=inputs.split("\n\n")
  print(inputs)
  sols = json.loads(ds[1]["solutions"])
  for sol in sols:
    case_outs=test_code(sols[0],inputs,2)
else:
  print("invalid")

invalid


In [ ]:
sols = json.loads(ds[1]["solutions"])
for sol in sols:
  case_outs=test_code(sols[0],inputs,2)